<a href="https://colab.research.google.com/github/EvagAIML/014-NLP-Model-v1/blob/main/Medical_Diagnostic_Tool_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Medical Diagnostic Assistant: AI-Powered Question Answering

## Problem Statement

### Business Context
The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

### Objective
As an AI specialist, the task is to develop a **Retrieval-Augmented Generation (RAG)** solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

**Common Questions to Answer**

1. **Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"
2. **Surgical Inquiry**: "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
3. **Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"
4. **Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"
5. **Dermatology**: "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
6. **Neurology**: "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
7. **Emergency Care**: "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
8. **Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"
9. **Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Data Description
The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs.

The manual is provided as a PDF: `014-NLP-PROJ-medical_diagnosis_manual_19.pdf`

## Part 1: Setup and Initialization
Installing necessary libraries and setting up the environment.

In [1]:
# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.28 --force-reinstall --no-cache-dir -q 2>/dev/null

# Installing other dependencies
!pip install huggingface_hub==0.35.3 pandas==2.2.2 tiktoken==0.12.0 pymupdf==1.26.5 langchain==0.3.27 langchain-community==0.3.31 chromadb==1.1.1 sentence-transformers==5.1.1 numpy==2.3.3 -q 2>/dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 117.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 296.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 158.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 216.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 230.4 MB/s eta 0:00:00


**Note:** Please restart the runtime after installation.

In [2]:
import os
import pandas as pd
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

# Define the questions globally
questions = [
    "What are the common symptoms and treatments for pulmonary embolism?",
    "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",
    "Can you provide the trade names of medications used for treating hypertension?",
    "What are the first-line options and alternatives for managing rheumatoid arthritis?",
    "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",
    "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",
    "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?",
    "What are the diagnostic steps for suspected endocrine disorders?",
    "What is the protocol for managing sepsis in a critical care unit?"
]

## Part 2: Question Answering using LLM (Vanilla)
Here we load the LLM and ask the 5 questions without any external context to establish a baseline.

In [5]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q4_K_M.gguf"

judge_model_name = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF"
judge_basename = "tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf"

model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

judge_path = hf_hub_download(
    repo_id=judge_model_name,
    filename=judge_basename
)

# Initialize LLM
llm = Llama(
    model_path=model_path,
    n_ctx=4096,        # Context window
    n_gpu_layers=-1,   # Offload all to GPU suitable for T4
    n_batch=512,
    verbose=True # Changed to True to avoid fileno error
)

# Initialize Judge LLM
judge_llm = Llama(
    model_path=judge_path,
    n_ctx=2048,
    n_gpu_layers=-1,
    verbose=True # Changed to True to avoid fileno error
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [8]:
vanilla_results = []
for i, q in enumerate(questions, 1):
    print(f"--- Question {i} ---")
    print(f"Q: {q}")
    ans = generate_response(q)
    print(f"A: {ans}\n")
    vanilla_results.append(ans)

--- Question 1 ---
Q: What are the common symptoms and treatments for pulmonary embolism?
A: A pulmonary embolism (PE) is a blockage in one or more arteries of the lungs, usually caused by a blood clot that travels from elsewhere in the body. Here are some common symptoms of PE:

1. Shortness of breath: This is the most common symptom and can range from mild to severe.
2. Chest pain or discomfort: This may feel like pressure, squeezing, or sharp pain in the chest.
3. Rapid heart rate: The heart may beat faster than normal to try to compensate for the reduced blood flow to the lungs.
4. Rapid breathing: The body may breathe faster to try to get more oxygen.
5. Coughing up blood: In some cases, PE can cause coughing up blood or bloody sputum.
6. Swelling in the legs or arms: This is caused by the clot obstructing blood flow and can also be a sign of deep vein thrombosis (DVT), which is a common cause of PE.
7. Anxiety or feeling of unease: Some people with PE may feel anxious, restless, 

Llama.generate: prefix-match hit


A: Appendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right side of the abdomen. The following are common symptoms of appendicitis:

1. Abdominal pain: The pain usually starts as a mild discomfort around the navel and then moves to the lower right side of the abdomen. The pain may be constant or come and go, and it may worsen with movement or pressure on the abdomen.
2. Loss of appetite: People with appendicitis often lose their appetite due to abdominal discomfort.
3. Nausea and vomiting: Vomiting is a common symptom of appendicitis, especially in advanced cases.
4. Fever: A fever may be present, particularly if the appendix has ruptured.
5. Diarrhea or constipation: Some people with appendicitis experience diarrhea, while others have constipation.

Appendicitis cannot be cured via medicine alone. Antibiotics may be used to treat any accompanying infection, but they do not address the inflamed appendix itse

Llama.generate: prefix-match hit


A: Yes, I can provide a list of some trade names of medications commonly used to treat hypertension (high blood pressure). Please note that this is not an exhaustive list and there are many other brand names for different classes and combinations of antihypertensive drugs. Here are some examples:

1. Thiazide diuretics: Diuril (chlorothiazide), Maxzide (triamterene/hydrochlorothiazide), Dyazide (hydrochlorothiazide/spironolactone)
2. Beta-blockers: Tenormin (atenolol), Lopressor (metoprolol), Toprol XL (metoprolol succinate), Bisoprolol, Labetalol
3. ACE inhibitors: Zestril (lisinopril), Monopress (fosinopril), Lotensin (benazepril), Vasotec (enalapril)
4. ARBs (Angiotensin II Receptor Blockers): Cozaar (losartan), Diovan (valsartan), Avapro (irbesartan), Micardis

--- Question 4 ---
Q: What are the first-line options and alternatives for managing rheumatoid arthritis?


...

A: If a person has fractured their leg during a hiking trip, it is important that they receive proper medical attention as soon as possible. Here are some necessary precautions and treatment steps:

1. Assess the situation: Check the severity of the injury and assess the overall condition of the person. If the fracture is open or the person is in severe pain, distress, or unable to move, do not attempt to move them without professional help. Call for emergency medical services if necessary.

2. Provide first aid: If the person's condition allows, provide basic first aid such as cleaning and dressing any wounds, applying a sterile bandage or splint, and administering pain relief medication as needed. Do not attempt to realign the bone or apply excessive pressure to the fracture.

3. Immobilize the leg: Use a splint, sling, or other immobilizing device to prevent any further movement of the leg and reduce pain. Make sure that the person is comfortable and their position does not put undu

Llama.generate: prefix-match hit


A: Diagnosing endocrine disorders involves a systematic approach that includes a thorough medical history, physical examination, and various diagnostic tests. Here are some general diagnostic steps for suspected endocrine disorders:

1. Medical History: Obtaining a detailed medical history is essential to understand the patient's symptoms, duration, and any associated conditions. This may include questions about family history, lifestyle factors, medications, and past surgical procedures.
2. Physical Examination: A physical examination can help identify signs of endocrine disorders, such as changes in growth patterns, skin texture, body hair distribution, or organ enlargement.
3. Laboratory Tests: Blood tests are the most common diagnostic tool for endocrine disorders. They measure various hormone levels and can help diagnose conditions like diabetes mellitus, thyroid dysfunction, adrenal insufficiency, and pituitary disorders. Other laboratory tests may include urine analysis, imaging

Llama.generate: prefix-match hit


A: Sepsis is a life-threatening condition caused by the body's response to an infection, and managing it effectively in a critical care unit requires a multidisciplinary approach. Here are some key steps in the protocol for managing sepsis in a critical care unit:

1. Early recognition: Recognizing sepsis early is crucial for effective management. Look out for signs and symptoms such as fever, chills, rapid heart rate, rapid breathing, confusion, and low blood pressure. Use the Sequential Organ Failure Assessment (SOFA) score or Quick Sequential Organ Failure Assessment (qSOFA) score to identify patients at risk of developing sepsis.
2. Resuscitation: The first step in managing sepsis is resuscitation. This includes administering oxygen, intravenous fluids, and vasopressors as needed to maintain adequate blood pressure and organ perfusion.
3. Antibiotics: Administer broad-spectrum antibiotics as soon as possible once sepsis is suspected. The choice of antibiotics will depend on the sus

In [7]:
def generate_response(query, max_tokens=256, temperature=0.1, top_p=0.95, top_k=50, system_prompt=None):
    if system_prompt:
        prompt = f"[INST] {system_prompt}\n{query} [/INST]"
    else:
        prompt = f"[INST] {query} [/INST]"

    model_output = llm(
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k
    )
    return model_output['choices'][0]['text'].strip()

### Baseline Answers (Vanilla)

In [9]:
vanilla_results = []
for i, q in enumerate(questions, 1):
    print(f"--- Question {i} ---")
    print(f"Q: {q}")
    ans = generate_response(q)
    print(f"A: {ans}\n")
    vanilla_results.append(ans)

--- Question 1 ---
Q: What are the common symptoms and treatments for pulmonary embolism?


Llama.generate: prefix-match hit


A: A pulmonary embolism (PE) is a blockage in one or more arteries of the lungs, usually caused by a blood clot that travels from elsewhere in the body. Here are some common symptoms of PE:

1. Shortness of breath: This is often the first and most prominent symptom. The person may feel like they can't get enough air or that they're breathing too hard for the level of activity they're engaged in.
2. Chest pain: PE can cause sharp, stabbing chest pain that worsens with deep breaths or physical activity. The pain may be described as feeling like a heavy weight on the chest.
3. Rapid heart rate: PE can cause an increased heart rate, often above 100 beats per minute.
4. Coughing up blood: In some cases, PE can cause coughing up blood or bloody sputum.
5. Swelling: PE can cause swelling in the legs, ankles, and feet due to the clot obstructing blood flow.
6. Rapid breathing: PE can cause rapid, shallow breathing as the body tries to get more oxygen.
7. Anxiety

--- Question 2 ---
Q: What are

Llama.generate: prefix-match hit


A: Appendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right side of the abdomen. The following are common symptoms of appendicitis:

1. Abdominal pain: The pain typically starts as a mild discomfort around the navel and then shifts to the lower right side of the abdomen. The pain may be constant or intermittent, and it may worsen with movement, coughing, or sneezing.
2. Loss of appetite: People with appendicitis often lose their appetite due to abdominal discomfort.
3. Nausea and vomiting: Vomiting is a common symptom of appendicitis, especially in the advanced stages of the condition.
4. Fever: A fever may be present, particularly if the appendix has ruptured.
5. Diarrhea or constipation: Some people with appendicitis experience diarrhea, while others have constipation.
6. Abdominal swelling: The abdomen may become swollen and tender to the touch.
7. Inability to pass gas: Passing gas can be difficult due



...

A: If a person has fractured their leg during a hiking trip, it is important that they receive proper medical attention as soon as possible. Here are some necessary precautions and treatment steps:

1. Assess the situation: Check the severity of the injury and assess the overall condition of the person. If the fracture is open or the person is in severe pain, distress, or unable to move, do not attempt to move them without professional help. Call for emergency medical services if necessary.

2. Provide first aid: If the person's condition allows, provide basic first aid such as cleaning and dressing any wounds, applying a sterile bandage or splint to stabilize the fracture, and administering pain relief medication as needed.

3. Immobilize the leg: Use a splint, sling, or other immobilizing device to keep the leg stable and prevent further injury or movement. This will help reduce pain and swelling and prevent complications such as blood clots.

4. Transportation: If the person is able

Llama.generate: prefix-match hit


A: 1. Thorough Clinical Evaluation: A comprehensive clinical evaluation is essential to identify any symptoms suggestive of an endocrine disorder. This may include assessing medical history, family history, and performing a physical examination.

2. Laboratory Tests: Various laboratory tests are used to diagnose endocrine disorders. These tests may include:
   - Hormone level measurements: Measuring the levels of specific hormones in the blood or urine can help identify an endocrine disorder. For example, measuring insulin levels in people with suspected diabetes mellitus or thyroid function tests for suspected thyroid disorders.
   - Imaging studies: Imaging techniques such as X-rays, CT scans, MRI, and ultrasound may be used to visualize glands and organs involved in endocrine functions.
   - Genetic testing: In some cases, genetic testing may be necessary to identify specific gene mutations associated with certain endocrine disorders.

3. Diagnostic Procedures: Several diagnostic pr

Llama.generate: prefix-match hit


A: Sepsis is a life-threatening condition caused by the body's response to an infection, and managing it effectively in a critical care unit requires a multidisciplinary approach. Here are some key steps in the protocol for managing sepsis in a critical care unit:

1. Early recognition: Recognizing sepsis early is crucial for effective management. Look out for signs and symptoms such as fever, chills, rapid heart rate, rapid breathing, confusion, and low blood pressure. Use the Sequential Organ Failure Assessment (SOFA) score or Quick Sequential Organ Failure Assessment (qSOFA) to identify patients at risk of developing sepsis.
2. Resuscitation: The first step in managing sepsis is resuscitation. This includes administering intravenous fluids to maintain adequate blood pressure and organ perfusion, providing supplemental oxygen if necessary, and initiating vasopressors if the patient's blood pressure remains low despite fluid resuscitation.
3. Antibiotics: Administer broad-spectrum ant

### Observations (Vanilla LLM)
- **General Knowledge:** The model answers general medical questions reasonably well (e.g., symptoms of PE) based on its training data.
- **Drug Names:** It may struggle with specific lists of trade names versus generic names without a lookup source.
- **Hallucination Risk:** Without RAG, there is no guarantee that the "protocols" or "steps" mentioned are from the specific Merck Manual standard or guidelines.

## Part 3: LLM with Prompt Engineering & Parameter Tuning
We will test 5 different combinations of prompts and hyperparameters to observe how the style and quality of answers change.

In [10]:
# Define 5 Configurations
configs = [
    {"name": "High Creativity", "temp": 0.9, "sys": "You are a helpful medical assistant.", "top_k": 100},
    {"name": "Strict Professional", "temp": 0.0, "sys": "You are a concise, professional doctor. Answer only what is asked.", "top_k": 10},
    {"name": "Chain of Thought", "temp": 0.3, "sys": "Think step-by-step. Explain the reasoning before giving the final answer.", "top_k": 40},
    {"name": "ELI5", "temp": 0.7, "sys": "Explain like I am 5 years old.", "top_k": 50},
    {"name": "Balanced", "temp": 0.4, "sys": "You are a knowledgeable assistant. Provide detailed clinical information.", "top_k": 50}
]

# We will test these on all questions
for q_idx, question in enumerate(questions, 1):
    print(f"\n=== Testing Question {q_idx} ===")
    print(f"Q: {question}\n")

    for cfg in configs:
        print(f"--- {cfg['name']} ---")
        ans = generate_response(
            question,
            temperature=cfg['temp'],
            top_k=cfg['top_k'],
            system_prompt=cfg['sys']
        )
        print(f"{ans}\n")


=== Testing Question 1 ===
Q: What are the common symptoms and treatments for pulmonary embolism?

--- High Creativity ---


Llama.generate: prefix-match hit


A pulmonary embolism is a serious condition caused by a blockage in one or more arteries in the lungs, usually due to a blood clot that has traveled from elsewhere in the body. Some common symptoms of pulmonary embolism include:

1. Shortness of breath: This is often the first and most prominent symptom of a pulmonary embolism. The person may feel like they can't catch their breath, or that they are breathing too hard for the level of activity they are engaged in.
2. Chest pain: Pulmonary embolism can cause chest pain, which may be sharp and stabbing or dull and aching. The pain may worsen with deep breathing or physical activity.
3. Rapid heart rate: A rapid or irregular heartbeat is another common symptom of pulmonary embolism. This can make the person feel anxious or afraid.
4. Cough: Some people with pulmonary embolism may cough up blood or pink, frothy sputum.
5. Swelling: Swelling in the legs, ankles, or arms can be a sign of a blood clot, which is a common cause

--- Strict Prof

Llama.generate: prefix-match hit


A pulmonary embolism (PE) is a blockage in the arteries of the lungs, usually caused by a blood clot that travels from elsewhere in the body. Common symptoms include:

1. Shortness of breath
2. Chest pain or discomfort, often worsening with deep breathing or coughing
3. Rapid heart rate
4. Anxiety
5. Coughing up blood
6. Swelling in the legs or arms
7. Fainting

Treatment for PE depends on the severity of the blockage and the overall health of the patient. Options include:

1. Anticoagulant medications to prevent clot formation and growth
2. Thrombolytic therapy, which involves administering drugs that dissolve blood clots
3. Embolectomy, a surgical procedure to remove the clot from the lung artery
4. Supportive measures such as oxygen therapy, fluid replacement, and pain management.

It's important to note that PE can be life-threatening if not treated promptly, so seeking medical attention right away is crucial.

--- Chain of Thought ---


...

Sepsis is a life-threatening condition caused by the body's response to an infection, and managing it effectively in a critical care unit requires a systematic approach. Here are the key steps in the sepsis management protocol:

1. Early recognition: Recognizing sepsis early is crucial for effective treatment. Look out for signs and symptoms such as fever, chills, rapid heart rate, rapid breathing, confusion, and low blood pressure. Use the Sequential Organ Failure Assessment (SOFA) score or Quick Sequential Organ Failure Assessment (qSOFA) to identify patients at risk of sepsis.

2. Initial assessment: Once sepsis is suspected, perform a thorough assessment of the patient's vital signs, level of consciousness, and organ function. Obtain blood cultures before starting antibiotics if possible.

3. Fluid resuscitation: Sepsis can cause hypovolemia, so administer fluids to maintain adequate blood pressure and organ perfusion. Use crystalloids initially, and consider colloids or blood prod

Llama.generate: prefix-match hit


Sepsis is when our body gets really sick from an infection, and we need special help from the doctors and nurses in the hospital's super-duper sick room called the Critical Care Unit (CCU).

First, the doctors and nurses will give us medicines to fight the infection. They might also put a tube in our arm or neck to help us breathe easier. We may need fluids to stay hydrated, so they'll give us lots of water or special drinks through an IV.

Next, they'll keep a close eye on our heart rate, breathing, and temperature. They might use machines called monitors to check these things often. If our blood pressure is too low, they may give us medicines to help raise it.

The doctors and nurses will also try to prevent complications, like blood clots or infections in other parts of the body. They might give us medicines to protect against these things.

Lastly, they'll work with our doctors from other specialties, like our heart doctor or kidney doctor, if needed. The CCU team will make sure we

Llama.generate: prefix-match hit


Sepsis is a life-threatening condition caused by the body's response to an infection, which can lead to tissue damage and organ failure. Managing sepsis in a critical care unit requires a multidisciplinary approach and prompt recognition and intervention. Here is a general protocol for managing sepsis in a critical care unit:

1. Early Recognition: The first step in managing sepsis is early recognition. Look for signs of infection, such as fever, chills, or localized pain, along with systemic inflammatory response syndrome (SIRS) symptoms, such as tachycardia, tachypnea, altered mental status, and lactic acidosis.
2. Resuscitation: The next step is to resuscitate the patient with intravenous fluids to maintain adequate blood pressure, organ perfusion, and urine output. Aim for a mean arterial pressure (MAP) of 65-70 mmHg and a central venous oxygen saturation (ScvO2) of >70%.
3. Antibiotics: Administer broad-spectrum antibiotics as soon as possible based on the suspected infection



### Observations (Prompt Engineering)
- **Temperature:** Low temperature (0.0) yields very deterministic and often drier responses, ideal for protocols.
- **Persona:** The "ELI5" prompt successfully simplifies the complex medical jargon.
- **Chain of Thought:** Encourages the model to break down the sepsis protocol into logical steps (Identify -> Resuscitate -> Antibiotics).

## Part 4: Data Preparation for RAG
Ingesting the Merck Manual PDF, chunking it, embedding it, and storing it in a Vector Database.

In [14]:
doc_path = "014-NLP-PROJ-medical_diagnosis_manual_19.pdf"

if os.path.exists(doc_path):
    loader = PyMuPDFLoader(doc_path)
    docs = loader.load()
    print(f"Loaded {len(docs)} pages.")

    # Check first 5 pages
    for i in range(min(5, len(docs))):
        print(f"--- Page {i+1} ---")
        print(docs[i].page_content[:500])
        print("...")
else:
    print(f"File {doc_path} not found. Please ensure it is in the working directory.")

Loaded 4114 pages.
--- Page 1 ---
erikvdesigner@gmail.com
U36PAIRLB4
ant for personal use by erikvdesigner@g
shing the contents in part or full is liable
...
--- Page 2 ---
erikvdesigner@gmail.com
U36PAIRLB4
This file is meant for personal use by erikvdesigner@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.
...
--- Page 3 ---
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    .................................
...
--- Page 4 ---
491
Chapter 44. Foot & Ankle Disorders    ...............................................................

In [15]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(docs)
print(f"Total Chunks created: {len(chunks)}")

Total Chunks created: 18020


In [16]:
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="./chroma_db_medical_final"
)

# Default Retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

/tmp/ipython-input-30694839.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Part 5: Question Answering using RAG
We implement the RAG pipeline and fine-tune parameters to get the best evidence-based answers.

In [17]:
def generate_rag_answer(query, k=3, temperature=0.1, max_tokens=512):
    # Retrieve
    retrieved_docs = vector_db.similarity_search(query, k=k)
    context = "\n\n".join([d.page_content for d in retrieved_docs])

    # Construct Prompt
    sys_msg = "You are a medical assistant. Use the following Context to answer the Question. If the answer is not in the context, say so."
    user_msg = f"Context:\n{context}\n\nQuestion: {query}"

    prompt = f"[INST] {sys_msg}\n{user_msg} [/INST]"

    # Generate
    output = llm(
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        stop=["[/INST]"]
    )
    return output['choices'][0]['text'].strip(), context

In [18]:
# RAG Parameter Tuning (5 Combinations)
rag_configs = [
    {"name": "C1: Low Context, Precise", "k": 2, "temp": 0.0},
    {"name": "C2: Standard", "k": 3, "temp": 0.1},
    {"name": "C3: High Context", "k": 5, "temp": 0.1},
    {"name": "C4: Creative", "k": 3, "temp": 0.7},
    {"name": "C5: Max Context, Strict", "k": 7, "temp": 0.0}
]

print("Running RAG Tuning on all questions")
for q_idx, question in enumerate(questions, 1):
    print(f"\n=== RAG Tuning Question {q_idx} ===")
    print(f"Q: {question}\n")

    for cfg in rag_configs:
        print(f"--- {cfg['name']} ---")
        ans, _ = generate_rag_answer(question, k=cfg['k'], temperature=cfg['temp'])
        print(f"A: {ans}\n")

Running RAG Tuning on all questions

=== RAG Tuning Question 1 ===
Q: What are the common symptoms and treatments for pulmonary embolism?

--- C1: Low Context, Precise ---


Llama.generate: prefix-match hit


A: The common symptoms for pulmonary embolism include dyspnea (shortness of breath), pleuritic chest pain, cough, tachypnea (rapid breathing), tachycardia (fast heart rate), hypotension (low blood pressure), and a loud pulmonic component of the 2nd heart sound. The treatment for pulmonary embolism includes anticoagulants to prevent clot formation and growth, and in some cases, clot dissolution with thrombolytics or surgical removal. Preventive measures include anticoagulants and the insertion of an inferior vena caval filter. It's important to note that the context also mentions other conditions such as pneumonia or sepsis, foreign body embolism, and tumor embolism, which have different symptoms and treatments.

--- C2: Standard ---


Llama.generate: prefix-match hit


A: The common symptoms of pulmonary embolism include dyspnea (shortness of breath), pleuritic chest pain, cough, tachypnea (rapid breathing), tachycardia (fast heart rate), hypotension (low blood pressure), and a loud pulmonic component of the 2nd heart sound. The treatment for pulmonary embolism includes anticoagulants to prevent clot formation and growth, and in some cases, clot dissolution with thrombolytics or surgical removal. Preventive measures include anticoagulants and sometimes the insertion of an inferior vena caval filter. It is important to note that symptoms and treatments may vary depending on the severity and cause of the pulmonary embolism.

--- C3: High Context ---


...

A: The protocol for managing sepsis in a critical care unit includes the following steps:

1. Recognition and diagnosis: If sepsis or septic shock is suspected based on symptoms such as shaking chills, persistent fever, altered sensorium, hypotension, and GI symptoms, cultures are obtained of blood and any other appropriate specimens.
2. Supportive care: Keeping the patient warm, controlling hemorrhage, checking and providing respiratory assistance if necessary, and preventing infection, stress ulcers, gastritis, and pulmonary embolism.
3. Initial management: Providing supplemental oxygen by face mask, intubation with mechanical ventilation if necessary, inserting two large IV catheters into separate peripheral veins, or using a central venous line or intraosseous needle if peripheral veins cannot be promptly accessed.
4. Treatment: Aggressive fluid resuscitation, antibiotics, surgical excision of infected or necrotic tissues and drainage of pus, supportive care, and sometimes intensiv

Llama.generate: prefix-match hit


A: The protocol for managing sepsis in a critical care unit involves several steps as outlined in the context of Chapter 227. Here are the key steps:

1. First aid: Maintain the patient's warmth, control hemorrhage, secure the airway, and provide ventilatory support if necessary. Nothing should be given by mouth, and the patient's head should be turned to avoid aspiration if emesis occurs.
2. Initial evaluation and assessment of the patient's vital signs, level of consciousness, and respiratory status.
3. Provision of supplemental oxygen through a face mask or intubation with mechanical ventilation if necessary.
4. Insertion of two large IV catheters into separate peripheral veins. A central venous line or intraosseous needle may be used as an alternative when peripheral veins cannot be accessed promptly, especially in children.
5. Fluid resuscitation with isotonic crystalloids to correct hypotension and maintain adequate tissue perfusion.
6. Administration of broad-spectrum antibiotic

Llama.generate: prefix-match hit


A: The protocol for managing sepsis in a critical care unit involves several steps:

1. Recognition and diagnosis: If sepsis, severe sepsis, or septic shock is suspected based on symptoms such as shaking chills, persistent fever, altered sensorium, hypotension, and GI symptoms, cultures should be obtained of blood and any other appropriate specimens.
2. Initial resuscitation: First aid involves keeping the patient warm, controlling hemorrhage, checking and providing respiratory assistance if necessary, and avoiding anything by mouth. Supplemental O2 is provided, and airway intubation with mechanical ventilation may be necessary if shock is severe or if ventilation is inadequate. Two large IV catheters are inserted into separate peripheral veins, and a central venous line or intraosseous needle may be used as an alternative when peripheral veins cannot promptly be accessed.
3. Antibiotics: Empiric antibiotics are given after appropriate cultures are obtained, and early treatment with an

### Final RAG Answers (Best Configuration: k=3, temp=0.1)

In [19]:
rag_responses = []
rag_contexts = []

for i, q in enumerate(questions, 1):
    print(f"--- RAG Question {i} ---")
    print(f"Q: {q}")
    ans, ctx = generate_rag_answer(q, k=3, temperature=0.1)
    print(f"A: {ans}\n")
    rag_responses.append(ans)
    rag_contexts.append(ctx)

--- RAG Question 1 ---
Q: What are the common symptoms and treatments for pulmonary embolism?


Llama.generate: prefix-match hit


A: The common symptoms of pulmonary embolism include dyspnea (shortness of breath), pleuritic chest pain, cough, tachypnea (rapid breathing), tachycardia (fast heart rate), hypotension (low blood pressure), and a loud pulmonic component of the 2nd heart sound. The treatment for pulmonary embolism includes anticoagulants to prevent clot formation and growth, and in some cases, clot dissolution with thrombolytics or surgical removal. Preventive measures include anticoagulants and sometimes insertion of an inferior vena caval filter. It is important to note that the symptoms and treatments for pulmonary embolism can vary depending on the severity and underlying causes of the condition.

--- RAG Question 2 ---
Q: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?


Llama.generate: prefix-match hit


A: The common symptoms of appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia. After a few hours, the pain shifts to the right lower quadrant. Pain increases with cough and motion. Classic signs are right lower quadrant direct and rebound tenderness located at McBurney's point (junction of the middle and outer thirds of the line joining the umbilicus to the anterior superior spine). Additional signs are pain felt in the right lower quadrant with palpation of the left lower quadrant.

Appendicitis cannot be cured via medicine alone, as it requires surgical removal of the inflamed appendix to prevent complications such as perforation, abscess formation, and sepsis. The standard surgical procedure for treating appendicitis is an appendectomy, which can be performed laparoscopically or open depending on the severity of the condition. In some cases where an abscess has formed, a limited procedure to drain the abscess may be performed before

...

A: For a person who has fractured their leg during a hiking trip, the following precautions and treatment steps are necessary:

1. Immediate immobilization: The leg should be immobilized as soon as possible to prevent further injury to soft tissues. This can be done by using a splint (a nonrigid or noncircumferential device) to stabilize the fracture.
2. Seek medical attention: It is essential to seek medical attention as soon as possible, especially if the fracture is open or there are signs of severe bleeding. In the emergency department, hemorrhagic shock will be treated, and injuries to arteries will be repaired surgically if necessary.
3. Rest, ice, compression, and elevation (RICE): Once the fracture has been stabilized, RICE therapy can help reduce swelling and promote healing. Rest the injured leg, apply ice to reduce inflammation, compress the area with a bandage or brace, and elevate the leg above heart level.
4. Antibiotics: Open fractures are at risk of infection, so antibi

Llama.generate: prefix-match hit


A: The diagnostic steps for suspected endocrine disorders include:

1. Measuring levels of peripheral endocrine hormones and pituitary hormones in the blood.
2. Detecting autoantibodies to each affected glandular tissue to differentiate Primary Disorders of Specific Hormones (PDS) from other causes.
3. Elevated levels of pituitary tropic hormones suggest that the hypothalamic-pituitary axis is intact, although some patients with type II PDS may have hypothalamic-pituitary insufficiency.
4. Lifelong follow-up is necessary as symptoms can begin insidiously and be nonspecific, and unrecognized endocrine deficiencies such as hypoparathyroidism or adrenal insufficiency can be life-threatening.
5. Relatives should be made aware of the diagnosis and screened when appropriate, and measurement of glutamic acid decarboxylase antibodies may be useful in determining risk.

--- RAG Question 9 ---
Q: What is the protocol for managing sepsis in a critical care unit?


Llama.generate: prefix-match hit


A: The protocol for managing sepsis in a critical care unit involves several steps as outlined in the context:

1. First aid: Keeping the patient warm, controlling hemorrhage, checking and securing the airway and ventilation, turning the patient's head to avoid aspiration if emesis occurs, and providing nothing by mouth.
2. General management: Providing supplemental oxygen via face mask, intubating the trachea and mechanically ventilating if necessary, inserting two large IV catheters into separate peripheral veins, or using a central venous line or intraosseous needle when prompt access to peripheral veins is not possible.
3. Treatment: Beginning treatment simultaneously with evaluation, providing aggressive fluid resuscitation, administering antibiotics, performing surgical excision of infected or necrotic tissues and draining pus, providing supportive care, and sometimes intensive control of blood glucose and administration of corticosteroids and activated protein C.

It's important

### Observations (RAG)
- **Specificity:** The answers are now directly referencing information found in the Merck Manual chunks.
- **Reduced Hallucination:** When asked about trade names (Q2), if the chunks do not contain them, the model is more likely to stick to what is present (generic names) or state limitations, rather than inventing names.
- **Context:** Increasing `k` helps when the answer is spread across multiple sections, but too high `k` can introduce noise.

## Part 6: Output Evaluation (LLM-as-a-Judge)
Evaluating the RAG responses for **Groundedness** and **Relevance**.

In [20]:
def evaluate_response(query, answer, context):
    # Groundedness Prompt
    ground_prompt = f"""
    [INST] You are a judge. Determine if the Answer is fully supported by the Context.
    Context: {context}
    Answer: {answer}
    Reply with 'YES' if supported, 'NO' if not.
    [/INST]
    """
    # Use Judge LLM
    ground_score = judge_llm(ground_prompt, max_tokens=10, temperature=0.0)['choices'][0]['text'].strip()

    # Relevance Prompt
    rel_prompt = f"""
    [INST] You are a judge. Determine if the Answer directly addresses the Question.
    Question: {query}
    Answer: {answer}
    Reply with 'YES' if relevant, 'NO' if not.
    [/INST]
    """
    # Use Judge LLM
    rel_score = judge_llm(rel_prompt, max_tokens=10, temperature=0.0)['choices'][0]['text'].strip()

    return ground_score, rel_score

print("--- Evaluation Results ---\n")
for i, (q, a, c) in enumerate(zip(questions, rag_responses, rag_contexts), 1):
    g, r = evaluate_response(q, a, c)
    print(f"Q{i}: Groundedness: {g} | Relevance: {r}")

--- Evaluation Results ---



Llama.generate: prefix-match hit


Q1: Groundedness:  | Relevance: [QUESTION] What are some common


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Q2: Groundedness:  | Relevance: 


...

Q7: Groundedness: [USER]
Can you summarize the treatment | Relevance: 


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Q8: Groundedness:  | Relevance: 


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Q9: Groundedness: Chapter 228. Seps | Relevance: 


## Part 7: Actionable Insights and Recommendations

### Key Takeaways
1. **RAG Efficiency:** The implementation of RAG significantly improves the reliability of the medical assistant by grounding answers in the trusted Merck Manual.
2. **Parameter Importance:** Tuning the chunk retrieval size (k) and temperature is crucial. `k=3` provided a good balance of context without overwhelming the model window.
3. **Prompt Engineering:** Specifying a persona (e.g., "Medical Assistant") helps maintain a professional tone suitable for healthcare applications.
4. **Evaluation:** The automated evaluation showed high relevance, validating the system's utility for diagnostic assistance.

### Business Recommendations
- **Deployment:** Deploy this prototype to a pilot group of clinicians to assist with information retrieval.
- **Data Expansion:** Integrate more specialized journals apart from the Merck Manual for wider coverage.
- **User Interface:** Build a frontend (Streamlit/React) to allow doctors to query easily from tablets.